In [102]:
import pandas as pd
import numpy as np

In [103]:
pd.set_option('display.max_rows', 200)

In [104]:
year19_path_list=  ['turnstile_190504.txt']

In [105]:
year17_path_list= ['turnstile_170506.txt', 'turnstile_170513.txt','turnstile_170527.txt', 'turnstile_170520.txt']

In [106]:
def load_data(list_of_datasets):
    year = pd.concat(map(lambda x: pd.read_csv(x,parse_dates=[['DATE','TIME']]), 
        list_of_datasets))
    return year

In [107]:
def preprocessing(dataframe):
    dataframe.reset_index(drop=True,inplace=True)
    dataframe.drop(['C/A','LINENAME','DIVISION', 'DESC'],axis=1,inplace = True)
    dataframe['UNIT/SCP']= dataframe['UNIT'] + dataframe['SCP'] #creating an ideantifier column 
    dataframe =  dataframe[(dataframe['DATE_TIME'].dt.month > 4)]
    return  dataframe
    

In [108]:
def calculate_entries_column(dataframe):
    dataframe.reset_index(drop=True,inplace=True)
    dataframe['ENTRIES_DIFF']=0
    for index, row in dataframe.iterrows():
        if index == 0:
            print('jhi')
            continue
        if row['UNIT/SCP'] == dataframe['UNIT/SCP'][index-1]:
            dataframe['ENTRIES_DIFF'][index] = row['ENTRIES'] - int(dataframe['ENTRIES'][index-1])
            
    return dataframe

In [109]:
year19=load_data(year19_path_list).head(100)

In [110]:
year19 = preprocessing(year19)

In [111]:
year19 = calculate_entries_column(year19)

jhi


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [112]:
year19

,DATE_TIME,UNIT,SCP,STATION,ENTRIES,EXITS,UNIT/SCP,ENTRIES_DIFF
0,2019-05-01 00:00:00,R051,02-00-00,59 ST,7039729,2386229,R05102-00-00,0
1,2019-05-01 04:00:00,R051,02-00-00,59 ST,7039743,2386234,R05102-00-00,14
2,2019-05-01 08:00:00,R051,02-00-00,59 ST,7039801,2386327,R05102-00-00,58
3,2019-05-01 12:00:00,R051,02-00-00,59 ST,7039966,2386534,R05102-00-00,165
4,2019-05-01 16:00:00,R051,02-00-00,59 ST,7040303,2386599,R05102-00-00,337
5,2019-05-01 20:00:00,R051,02-00-00,59 ST,7041075,2386673,R05102-00-00,772
6,2019-05-02 00:00:00,R051,02-00-00,59 ST,7041303,2386712,R05102-00-00,228
7,2019-05-02 04:00:00,R051,02-00-00,59 ST,7041318,2386714,R05102-00-00,15
8,2019-05-02 08:00:00,R051,02-00-00,59 ST,7041356,2386810,R05102-00-00,38
9,2019-05-02 12:00:00,R051,02-00-00,59 ST,7041532,2387060,R05102-00-00,176


In [38]:
year17=load_data(year17_path_list)

In [39]:
year17 = preprocessing(year17)